In [20]:
require 'daru'
require 'daru_plotting_plotly'
require 'rbplotly'
require 'hg'

Daru.plotting_library = :plotly

class Numeric
  def mili
    (self * 10 ** (-3)).to_f
  end
  
  def micro
    (self * 10 ** (-6)).to_f
  end
end

class LeastSquareFitting
  def initialize(xs, ys)
    raise ArgumentError "xs and ys should be same size" if xs.size != ys.size
    @xs = xs
    @ys = ys
  end
  
  def fit
    xy = @xs.zip(@ys).map { |x, y| x * y }.sum
    x = @xs.sum
    y = @ys.sum
    x2 = @xs.sum { |x| x ** 2 }
    n = @xs.size
    
    @a = (n*xy - x*y)/(n*x2 - x**2)
    @b = (x2*y - xy*x)/(n*x2 - x**2)
  end
  
  attr_reader :a, :b
end

class ExpFitting < LeastSquareFitting
  def initialize(xs, ys)
    super(xs, ys.map{ |y| Math.log(y) })
  end
  
  def to_s
    "log y = #{'%1.2e' % @a} x #{ @b.positive? ? '+' : '-' } #{'%1.2e' % @b.abs }"
  end
  
  def points(min, max)
    ys = [min, max].map { |x| Math.exp(@b) * Math.exp( @a * x ) }
    { x: [min, max], y: ys, name: to_s }
  end
end

class LinearFitting < LeastSquareFitting
  def points(min, max)
    ys = [min, max].map { |x| x * @a + @b }
    { x: [min, max], y: ys , name: to_s}
  end
  
  def to_s
    "y = #{'%1.2f' % @a} x + #{'%1.2f' % @b}"
  end
end

:to_s

In [21]:
def map_color(x)
  if x == 'yellow'
    'orange'
  elsif x == 'white'
    'black'
  else
    x
  end
end

['red.csv', 'yellow.csv', 'blue.csv', 'white.csv', 'green.csv'].map { |name| [name, Daru::DataFrame.from_csv(name)] }.map do |name, df|
  color = File.basename(name, '.csv')
  {
    name: color,
    x: df['v'].to_a,
    y: df['i'].to_a,
    type: :scatter,
    marker: { color: map_color(color) },
    mode: 'markers+lines'
  }
end.tap do |data|
  Plotly::Plot.new(data: data).show
  data.zip([[1, 1.63],[1.6,1.9],[2.4, 2.74],[2.3, 2.65],[2,2.85]]).each do |trace, range|
    x, y = trace[:x].zip(trace[:y]).select { |x, _| range[0] <= x && x <= range[1] }.transpose
    fitting = ExpFitting.new(x, y)
    fitting.fit
    fit_line = fitting.points(trace[:x].min, trace[:x].max)
    Plotly::Plot.new(data: [trace, fit_line], layout: {yaxis: { type: :log }}).show
  end
end
nil

#<Plotly::Offline::HTML:0x007fb9353e0648 @id="7252e8a4-3fba-4875-8486-24a0bf8a50de", @data=[{:name=>"red", :x=>[0.54, 1.076, 1.629, 1.72, 1.78, 1.863, 1.934, 1.996], :y=>[1.0e-14, 2.0e-14, 6.0e-06, 0.00068, 0.0023, 0.006, 0.011, 0.0146], :type=>:scatter, :marker=>{:color=>"red"}, :mode=>"markers+lines"}, {:name=>"yellow", :x=>[1.62, 1.768, 1.817, 1.897, 1.945, 1.983, 2.01], :y=>[2.8e-05, 0.000505, 0.0013000000000000002, 0.004900000000000001, 0.0087, 0.012400000000000001, 0.0162], :type=>:scatter, :marker=>{:color=>"orange"}, :mode=>"markers+lines"}, {:name=>"blue", :x=>[3.15, 3.13, 3.1, 3.06, 3.04, 3.0, 2.96, 2.92, 2.89, 2.84, 2.8, 2.77, 2.73, 2.66, 2.64, 2.62, 2.6, 2.6, 2.59, 2.56, 2.55, 2.54, 2.52, 2.48, 2.46, 2.42, 2.34], :y=>[0.0155, 0.0143, 0.0121, 0.01, 0.0089, 0.0076, 0.006, 0.00474, 0.0037, 0.00272, 0.00202, 0.00149, 0.00102, 0.000415, 0.000336, 0.000262, 0.000188, 0.00017, 0.000146, 0.000102, 8.7e-05, 7.5e-05, 5.5e-05, 3.2e-05, 2.2e-05, 1.2e-05, 5.0e-06], :type=>:scatter, :marker=>{:color=>"blue"}, :mode=>"markers+lines"}, {:name=>"white", :x=>[3.23, 3.03, 2.83, 2.74, 2.61, 2.5, 2.33, 0.852], :y=>[0.0154, 0.0075, 0.0026000000000000003, 0.0013000000000000002, 0.00038, 6.0e-05, 1.0e-06, 8.0e-08], :type=>:scatter, :marker=>{:color=>"black"}, :mode=>"markers+lines"}, {:name=>"green", :x=>[3.11, 2.96, 2.82, 2.73, 2.59, 2.04, 1.5], :y=>[0.0156, 0.008400000000000001, 0.0034700000000000004, 0.00161, 0.000388, 1.0e-06, 1.0e-07], :type=>:scatter, :marker=>{:color=>"green"}, :mode=>"markers+lines"}], @layout={}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb9353c0f50 @id="09a3d142-0198-4221-8655-c98694bb8955", @data=[{:name=>"red", :x=>[0.54, 1.076, 1.629, 1.72, 1.78, 1.863, 1.934, 1.996], :y=>[1.0e-14, 2.0e-14, 6.0e-06, 0.00068, 0.0023, 0.006, 0.011, 0.0146], :type=>:scatter, :marker=>{:color=>"red"}, :mode=>"markers+lines"}, {:name=>"log y = 3.53e+01 x - 6.95e+01", :x=>[0.54, 1.996], :y=>[1.214807334083864e-22, 2.535225132737849]}], @layout={:yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb9353a3e00 @id="3c7bf807-a42a-4043-8af2-88e7075aebd2", @data=[{:name=>"yellow", :x=>[1.62, 1.768, 1.817, 1.897, 1.945, 1.983, 2.01], :y=>[2.8e-05, 0.000505, 0.0013000000000000002, 0.004900000000000001, 0.0087, 0.012400000000000001, 0.0162], :type=>:scatter, :marker=>{:color=>"orange"}, :mode=>"markers+lines"}, {:name=>"log y = 1.88e+01 x - 4.09e+01", :x=>[1.62, 2.01], :y=>[2.9497223219020442e-05, 0.044889649262136774]}], @layout={:yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb934ce4e98 @id="a245ea71-74bd-429e-accf-cd2b10f7b397", @data=[{:name=>"blue", :x=>[3.15, 3.13, 3.1, 3.06, 3.04, 3.0, 2.96, 2.92, 2.89, 2.84, 2.8, 2.77, 2.73, 2.66, 2.64, 2.62, 2.6, 2.6, 2.59, 2.56, 2.55, 2.54, 2.52, 2.48, 2.46, 2.42, 2.34], :y=>[0.0155, 0.0143, 0.0121, 0.01, 0.0089, 0.0076, 0.006, 0.00474, 0.0037, 0.00272, 0.00202, 0.00149, 0.00102, 0.000415, 0.000336, 0.000262, 0.000188, 0.00017, 0.000146, 0.000102, 8.7e-05, 7.5e-05, 5.5e-05, 3.2e-05, 2.2e-05, 1.2e-05, 5.0e-06], :type=>:scatter, :marker=>{:color=>"blue"}, :mode=>"markers+lines"}, {:name=>"log y = 1.45e+01 x - 4.64e+01", :x=>[2.34, 3.15], :y=>[4.046700189595398e-06, 0.5154100238719347]}], @layout={:yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb934c95208 @id="3aedcba1-8fd2-4552-b3f4-62cdc1ee5902", @data=[{:name=>"white", :x=>[3.23, 3.03, 2.83, 2.74, 2.61, 2.5, 2.33, 0.852], :y=>[0.0154, 0.0075, 0.0026000000000000003, 0.0013000000000000002, 0.00038, 6.0e-05, 1.0e-06, 8.0e-08], :type=>:scatter, :marker=>{:color=>"black"}, :mode=>"markers+lines"}, {:name=>"log y = 2.15e+01 x - 6.37e+01", :x=>[0.852, 3.23], :y=>[1.9042775524057968e-20, 277.09205932173865]}], @layout={:yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<Plotly::Offline::HTML:0x007fb934c5ccc8 @id="ba7f960b-a3de-4240-a644-2a8072a75fb3", @data=[{:name=>"green", :x=>[3.11, 2.96, 2.82, 2.73, 2.59, 2.04, 1.5], :y=>[0.0156, 0.008400000000000001, 0.0034700000000000004, 0.00161, 0.000388, 1.0e-06, 1.0e-07], :type=>:scatter, :marker=>{:color=>"green"}, :mode=>"markers+lines"}, {:name=>"log y = 1.06e+01 x - 3.53e+01", :x=>[1.5, 3.11], :y=>[3.463362705104535e-09, 0.08437965461386107]}], @layout={:yaxis=>{:type=>:log}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>